In [11]:
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

In [75]:
#Clear the previous tensorflow graph
K.clear_session()

FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    #model.add(Dense(5, activation='relu', kernel_initializer='glorot_normal', name='fc3'))
    model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='tanh', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

In [76]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

windowSize = 25
stride = 2


dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_1(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = 105)
tModel.loadData(verbose=1)

Loading data for dataset 1 with window_size of 25, stride of 2 and constRUL of 105. Cros-Validation ratio 0
Data loaded for dataset 1


In [80]:
#tModel.printData(printTop=False)
#tModel.windowSize = 20
#tModel.changeDataset(2)
#tModel.loadData(verbose=1, crossValRatio=0.1)
#tModel.printData(printTop=False)
tModel.getModelDescription()
ruldcnn = RULCNNModel(tModel.windowSize, len(selected_features))
ruldcnn.summary()

Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 30)                10530     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                310       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
out (Dense)                  (None, 1)                 11        
Total params: 10,851
Trainable params: 10,851
Non-trainable params: 0
_________________________________________________________________


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C1", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C2", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C3", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:65: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glo

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 25, 14)            0         
_________________________________________________________________
Reshape (Reshape)            (None, 25, 14, 1)         0         
_________________________________________________________________
C1 (Conv2D)                  (None, 25, 14, 10)        110       
_________________________________________________________________
C2 (Conv2D)                  (None, 25, 14, 10)        1010      
_________________________________________________________________
C3 (Conv2D)                  (None, 25, 14, 10)        1010      
_________________________________________________________________
C4 (Conv2D)                  (None, 25, 14, 10)        1010      
_________________________________________________________________
Clast (Conv2D)               (None, 25, 14, 1)         31        
__________

In [81]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
#tModel.setCurrentModel('RULmodel_SN_1')
#tModel.getModelDescription()
tModel.epochs = 250
tModel.trainModel(learningRateScheduler=lrate, verbose=1)

Epoch 1/250
9141/9141 [==============================] - 0s 32us/step - loss: 775.2247 - mean_squared_error: 775.2247
Epoch 2/250
9141/9141 [==============================] - 0s 10us/step - loss: 795.6086 - mean_squared_error: 795.6086
Epoch 3/250
9141/9141 [==============================] - 0s 13us/step - loss: 794.9999 - mean_squared_error: 794.9999 0s - loss: 793.2303 - mean_squared_error: 793.23
Epoch 4/250
9141/9141 [==============================] - 0s 13us/step - loss: 779.0827 - mean_squared_error: 779.0827
Epoch 5/250
9141/9141 [==============================] - 0s 14us/step - loss: 765.1051 - mean_squared_error: 765.1051
Epoch 6/250
9141/9141 [==============================] - 0s 12us/step - loss: 793.1828 - mean_squared_error: 793.1828
Epoch 7/250
9141/9141 [==============================] - 0s 11us/step - loss: 773.4916 - mean_squared_error: 773.4916
Epoch 8/250
9141/9141 [==============================] - 0s 11us/step - loss: 777.0780 - mean_squared_error: 777.0780
Epoch 9

9141/9141 [==============================] - 0s 15us/step - loss: 673.5956 - mean_squared_error: 673.5956
Epoch 70/250
9141/9141 [==============================] - 0s 12us/step - loss: 663.2593 - mean_squared_error: 663.2593
Epoch 71/250
9141/9141 [==============================] - 0s 11us/step - loss: 658.0952 - mean_squared_error: 658.0952
Epoch 72/250
9141/9141 [==============================] - 0s 12us/step - loss: 671.2878 - mean_squared_error: 671.2878
Epoch 73/250
9141/9141 [==============================] - 0s 12us/step - loss: 659.3602 - mean_squared_error: 659.3602
Epoch 74/250
9141/9141 [==============================] - 0s 11us/step - loss: 668.0946 - mean_squared_error: 668.0946
Epoch 75/250
9141/9141 [==============================] - 0s 11us/step - loss: 649.5046 - mean_squared_error: 649.5046
Epoch 76/250
9141/9141 [==============================] - 0s 11us/step - loss: 663.8776 - mean_squared_error: 663.8776
Epoch 77/250
9141/9141 [==============================] - 0s 

9141/9141 [==============================] - 0s 10us/step - loss: 553.8699 - mean_squared_error: 553.8699
Epoch 138/250
9141/9141 [==============================] - 0s 9us/step - loss: 552.3663 - mean_squared_error: 552.3663
Epoch 139/250
9141/9141 [==============================] - 0s 9us/step - loss: 555.5884 - mean_squared_error: 555.5884
Epoch 140/250
9141/9141 [==============================] - 0s 9us/step - loss: 557.0877 - mean_squared_error: 557.0877
Epoch 141/250
9141/9141 [==============================] - 0s 9us/step - loss: 546.4831 - mean_squared_error: 546.4831
Epoch 142/250
9141/9141 [==============================] - 0s 10us/step - loss: 544.4243 - mean_squared_error: 544.4243
Epoch 143/250
9141/9141 [==============================] - 0s 9us/step - loss: 556.3605 - mean_squared_error: 556.3605
Epoch 144/250
9141/9141 [==============================] - 0s 9us/step - loss: 543.7642 - mean_squared_error: 543.7642
Epoch 145/250
9141/9141 [==============================] - 0

9141/9141 [==============================] - 0s 10us/step - loss: 510.6117 - mean_squared_error: 510.6117
Epoch 206/250
9141/9141 [==============================] - 0s 9us/step - loss: 501.6724 - mean_squared_error: 501.6724
Epoch 207/250
9141/9141 [==============================] - 0s 9us/step - loss: 487.9619 - mean_squared_error: 487.9619
Epoch 208/250
9141/9141 [==============================] - 0s 10us/step - loss: 511.8940 - mean_squared_error: 511.8940
Epoch 209/250
9141/9141 [==============================] - 0s 9us/step - loss: 496.0837 - mean_squared_error: 496.0837
Epoch 210/250
9141/9141 [==============================] - 0s 9us/step - loss: 504.8620 - mean_squared_error: 504.8620
Epoch 211/250
9141/9141 [==============================] - 0s 9us/step - loss: 502.9380 - mean_squared_error: 502.9380
Epoch 212/250
9141/9141 [==============================] - 0s 10us/step - loss: 499.3304 - mean_squared_error: 499.3304
Epoch 213/250
9141/9141 [==============================] - 

In [82]:
tModel.evaluateModel(["rhs"])
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rhs = cScores['rhs']
time = tModel.trainTime

print(cScores)
print("RMSE: {}".format(rmse))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 94us/step
scores
{'loss': 128.8965106201172, 'score_1': 128.8965106201172, 'rhs': 212.25792918191237}
RMSE: 11.353259911590028
RHS: 212.25792918191237
Time : 38.10572400000001 seconds


In [ ]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"], crossValidation=True)
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse

In [ ]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    
    maxCycle = X_test2.shape[0]
    faultCycle = tModel.y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()

In [ ]:
plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, stride, constRUL)